# Meta Engines

In this notebook we define an oversubscription planning problem and we solve it using a `MetaEngine`.

### Setup the library

First, we install unified_planning library and its dependencies from PyPi. Here, we use the `--pre` flag to use the latest development build.

In [ ]:
# begin of installation

In [ ]:
pip install --pre unified-planning[tamer,pyperplan]

In [ ]:
# end of installation

### Problem definition

We model an oversubscription planning problem.

In [ ]:
from unified_planning.shortcuts import *

We start the problem modeling defining the `UserType` and the `Fluent`.

In [ ]:
Location = UserType('Location')
Robot = UserType('Robot')

at = Fluent('at', BoolType(), robot=Robot, location=Location)
connected = Fluent('connected', BoolType(), l_from=Location, l_to=Location)

We define an action `move` that models the movement of a robot between two locations.


In [ ]:
move = InstantaneousAction('move', robot=Robot, l_from=Location, l_to=Location)
robot = move.parameter('robot')
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(at(robot, l_from))
move.add_precondition(connected(l_from, l_to))
move.add_effect(at(robot, l_from), False)
move.add_effect(at(robot, l_to), True)

We define the `Object` instances and, after creating the `Problem`, we set the initial values.

In [ ]:
r1 = Object('r1', Robot)
NLOC = 10
locations = [Object('l%s' % i, Location) for i in range(NLOC)]

problem = Problem('robot_with_simulated_effects')
problem.add_fluent(at, default_initial_value=False)
problem.add_fluent(connected, default_initial_value=False)
problem.add_action(move)

problem.add_object(r1)
problem.add_objects(locations)

problem.set_initial_value(at(r1, locations[0]), True)
for i in range(NLOC - 1):
    problem.set_initial_value(connected(locations[i], locations[i+1]), True)
problem.set_initial_value(connected(locations[4], locations[8]), True)

Finally, we define the oversubscription goals.

In [ ]:
goals = {}
goals[at(r1, locations[5])] = 5
goals[at(r1, locations[7])] = 4
goals[at(r1, locations[9])] = 10

problem.add_quality_metric(up.model.metrics.Oversubscription(goals))

### Solving the problem

We solve the problem using the oversubscription `MetaEngine` with the tamer `Engine`.

In [ ]:
with OneshotPlanner(name='oversubscription[tamer]') as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))

But the same `MetaEngine` can be used with other `Engine`, then we re-solve the problem with the pyperplan `Engine`.

In [ ]:
with OneshotPlanner(name='oversubscription[pyperplan]') as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))

To test the oversubscription `MetaEngine`, we update the oversubscription goals to see if it finds a different plan.

In [ ]:
problem.clear_quality_metrics()

goals = {}
goals[at(r1, locations[5])] = 5
goals[at(r1, locations[7])] = 6
goals[at(r1, locations[9])] = 10

problem.add_quality_metric(up.model.metrics.Oversubscription(goals))

To solve the new problem, now we let the system choose the `Engine` to use.

In [ ]:
with OneshotPlanner(problem_kind=problem.kind) as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))